In [24]:
import sys
# sys.path.append("Codes/")
# import TMPD_understanding_tasks.py

In [35]:
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest

# Load data from the Excel file
reference_window_df = pd.read_excel("Input/Test/GPT_ConceptDrift_Samples.xlsx", sheet_name='Reference Window')
detection_window_df = pd.read_excel("Input/Test/GPT_ConceptDrift_Samples.xlsx", sheet_name='Detection Window')

# Helper function to identify the type of variable
def identify_variable_type(series):
    """
    Identify the type of variable based on its values.
    - 'proportion' if values are between 0 and 1.
    - 'continuous' otherwise.
    """
    if series.between(0, 1).all():
        return 'proportion'
    return 'continuous'

def perform_z_test_for_proportions(reference_counts, detection_counts, reference_nobs, detection_nobs):
    """
    Conduct a Z-test for two proportions.
    
    Parameters:
    - reference_counts: Number of successes in the reference dataset.
    - detection_counts: Number of successes in the detection dataset.
    - reference_nobs: Number of observations in the reference dataset.
    - detection_nobs: Number of observations in the detection dataset.

    Returns:
    - p-value of the Z-test.
    """
    stat, pval = proportions_ztest([reference_counts, detection_counts], [reference_nobs, detection_nobs])
    return pval

# Generic function to detect statistically significant transitions
def detect_significant_transitions(reference_df, detection_df):
    """
    Detect transitions with statistically significant differences for all variables.
    """
    significant_transitions = []

    # Merge the two dataframes based on 'From' and 'To' columns
    merged_df = pd.merge(reference_df, detection_df, on=['From', 'To'], how='outer', suffixes=('_ref', '_det'))
    merged_df.fillna(0, inplace=True)  # Replace NaN values with 0

    # Identify variables to test (excluding 'From', 'To', and 'Frequency')
    test_vars = [col for col in merged_df.columns if col not in ['From', 'To', 'Frequency_ref', 'Frequency_det']]
    test_vars = [var.split('_ref')[0] for var in test_vars if '_ref' in var]

    for var in test_vars:
        var_type = identify_variable_type(merged_df[f'{var}_ref'])

        # Test for statistical significance based on the variable type
        if var_type == "proportion":
            merged_df[f'p_value_{var}'] = merged_df.apply(lambda row: perform_z_test_for_proportions(
                row[f'{var}_ref'] * row['Frequency_ref'], 
                row[f'{var}_det'] * row['Frequency_det'], 
                row['Frequency_ref'], 
                row['Frequency_det']
            ), axis=1)
        else:  # continuous
            stat, pval = ttest_ind(
                merged_df[f'{var}_ref'] * merged_df['Frequency_ref'], 
                merged_df[f'{var}_det'] * merged_df['Frequency_det'], 
                equal_var=False, nan_policy='omit'
            )
            merged_df[f'p_value_{var}'] = pval
        
        # Filter for significant transitions
        sig_transitions = merged_df[merged_df[f'p_value_{var}'] < 0.05][['From', 'To', f'{var}_ref', f'{var}_det', f'p_value_{var}']]
        significant_transitions.append(sig_transitions)

    return significant_transitions

# Detect significant transitions for all variables in the datasets
significant_transitions_list = detect_significant_transitions(reference_window_df, detection_window_df)
significant_transitions_list

d:\OneDrive\_Cloud Disk\Projetos de programação\Python Projects\Git\GitHub\Transition_Matrix_Process_Drift\.venv\Lib\site-packages\statsmodels\stats\proportion.py:1005: RuntimeWarning: invalid value encountered in divide
  prop = count * 1. / nobs
d:\OneDrive\_Cloud Disk\Projetos de programação\Python Projects\Git\GitHub\Transition_Matrix_Process_Drift\.venv\Lib\site-packages\statsmodels\stats\proportion.py:1019: RuntimeWarning: invalid value encountered in scalar divide
  p_pooled = np.sum(count) * 1. / np.sum(nobs)
d:\OneDrive\_Cloud Disk\Projetos de programação\Python Projects\Git\GitHub\Transition_Matrix_Process_Drift\.venv\Lib\site-packages\statsmodels\stats\proportion.py:1021: RuntimeWarning: divide by zero encountered in divide
  nobs_fact = np.sum(1. / nobs)
d:\OneDrive\_Cloud Disk\Projetos de programação\Python Projects\Git\GitHub\Transition_Matrix_Process_Drift\.venv\Lib\site-packages\statsmodels\stats\weightstats.py:792: RuntimeWarning: invalid value encountered in scalar di

TypeError: '>=' not supported between instances of 'str' and 'int'

In [ ]:
from scipy.stats import fisher_exact

# Initialize a list to store the results of the statistical tests
significant_transitions = []

# Loop through each unique transition and perform the appropriate statistical test
for transition in all_unique_transitions_list:
    # Extract the frequencies for the transition in both sheets
    freq_reference = reference_window_df.loc[(reference_window_df['From'] == transition[0]) & 
                                             (reference_window_df['To'] == transition[1]), 'Frequency'].sum()
    freq_detection = detection_window_df.loc[(detection_window_df['From'] == transition[0]) & 
                                             (detection_window_df['To'] == transition[1]), 'Frequency'].sum()
    
    # Construct the contingency table
    contingency_table = [[freq_reference, freq_detection],
                         [total_freq_reference - freq_reference, total_freq_detection - freq_detection]]
    
    # Calculate the minimum expected frequency in the contingency table
    row_totals = [sum(row) for row in contingency_table]
    col_totals = [sum(col) for col in zip(*contingency_table)]
    grand_total = sum(row_totals)
    expected_min = min((row_total * col_total / grand_total) for row_total, col_total in zip(row_totals, col_totals))
    
    # Perform the Chi-Square test if the minimum expected frequency is 5 or more, else perform Fisher's Exact Test
    if expected_min >= 5:
        _, p, _, _ = chi2_contingency(contingency_table, correction=False)
    else:
        _, p = fisher_exact(contingency_table)
    
    # If p-value is less than 0.05, then the transition is considered as having a statistically significant difference
    if p < 0.05:
        significant_transitions.append({
            'transition': transition,
            'p_value': p,
            'freq_reference': freq_reference,
            'freq_detection': freq_detection
        })

# Display the transitions with statistically significant differences
significant_transitions[:5], len(significant_transitions)

In [25]:
import os
from langchain.llms import OpenAI
from langchain.agents import create_pandas_dataframe_agent 
import pandas as pd

# insert API_TOKEN in the file to be read here
with open('Temp/open_ai_token.txt', 'r') as file:
    os.environ["OPENAI_API_KEY"] = file.read().rstrip()


In [22]:
# Instanciating a proprietary LLM from OpenAI
llm = OpenAI(model_name="text-davinci-003", temperature=0)

In [30]:
# The LLM takes a prompt as an input and outputs a completion
# prompt = "Alice has a parrot. What animal is Alice's pet?"
# completion = llm(prompt)
# print(completion)

In [33]:
agent = create_pandas_dataframe_agent(llm, reference_window, verbose=True)
agent.run("Summary what have in the dataset")




> Entering new AgentExecutor chain...
Thought: I need to know what columns are present and what is the content of them
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['From', 'To', 'Probability', 'Frequency', 'Text format'], dtype='object')
Thought:

d:\OneDrive\_Cloud Disk\Projetos de programação\Python Projects\Git\GitHub\Transition_Matrix_Process_Drift\.venv\Lib\site-packages\langchain\tools\python\tool.py:137: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(


 I now need to know the data types of the columns
Action: python_repl_ast
Action Input: df.dtypes
Observation: From            object
To              object
Probability    float64
Frequency        int64
Text format     object
dtype: object
Thought:

d:\OneDrive\_Cloud Disk\Projetos de programação\Python Projects\Git\GitHub\Transition_Matrix_Process_Drift\.venv\Lib\site-packages\langchain\tools\python\tool.py:137: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(


 I now need to know the data summary
Action: python_repl_ast
Action Input: df.describe()

d:\OneDrive\_Cloud Disk\Projetos de programação\Python Projects\Git\GitHub\Transition_Matrix_Process_Drift\.venv\Lib\site-packages\langchain\tools\python\tool.py:137: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(



Observation:        Probability   Frequency
count    63.000000   63.000000
mean      0.126984   12.698413
std       0.283530   28.352980
min       0.000000    0.000000
25%       0.000000    0.000000
50%       0.000000    0.000000
75%       0.000000    0.000000
max       1.000000  100.000000
Thought: I now know the final answer
Final Answer: The dataframe contains five columns of data, with the columns 'From' and 'To' as strings, 'Probability' and 'Frequency' as numeric values, and 'Text format' as a string. The summary statistics show that the Probability ranges from 0 to 1, and the Frequency ranges from 0 to 100.

> Finished chain.


"The dataframe contains five columns of data, with the columns 'From' and 'To' as strings, 'Probability' and 'Frequency' as numeric values, and 'Text format' as a string. The summary statistics show that the Probability ranges from 0 to 1, and the Frequency ranges from 0 to 100."